In [1]:
#basics -> imports and apikey
import json
import requests as rq
import pandas as pd
import time

apikey = '9c594b29-ad69-4833-b88b-db5cdfa30683'

In [2]:
list_summoners = pd.read_excel('LVPSummoner.xlsx',
                      sheetname = 'table_output',
                      header = 0,
                      index_col = 0,
                      parse_cols = "A, B",
                      convert_float = True)

#check what was actually extracted into the table
print list_summoners.head()
print list_summoners.count()

                  A_Summoner
Summoner ID                 
19459077           nuskineta
20501992     huevodestructor
23407188           pitterman
27281117             ajrking
28306135      marcodiazlocos
A_Summoner    440
dtype: int64


In [3]:
list_summoners.index[0]

19459077

In [4]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [5]:
db = client.lol_match_db
collection = db.match_list_2016_5x5

In [31]:
## Query MATCHLIST
#https://euw.api.pvp.net/api/lol/euw/v2.2/matchlist/by-summoner/230016?rankedQueues=
#RANKED_SOLO_5x5,RANKED_TEAM_5x5&seasons=PRESEASON2015,SEASON2015,PRESEASON2016,SEASON2016
#&api_key=9c594b29-ad69-4833-b88b-db5cdfa30683

## Query MATCH
#https://euw.api.pvp.net/api/lol/euw/v2.2/match/2216416155?includeTimeline=TRUE&api_key=9c594b29-ad69-4833-b88b-db5cdfa30683


In [65]:
len(list_summoners.index)

440

In [7]:
import sys
filename  = open("log_match_history_2016.txt",'w')
sys.stdout = filename
print "RESPONSES"

In [8]:
#test with i=0
result = db.match_list_2016.remove({ "_id": str(list_summoners.index[0])})
r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/matchlist/by-summoner/' +str(list_summoners.index[0])+
           '?rankedQueues=TEAM_BUILDER_DRAFT_RANKED_5x5,RANKED_SOLO_5x5,RANKED_TEAM_5x5,RANKED_FLEX_SR&seasons=PRESEASON2016,SEASON2016&api_key=' + apikey)
#check response code 
r.status_code

200

In [9]:
list_summoners.index[0]

19459077

In [10]:
(r.status_code == 200)

True

In [11]:
#insert into database
###result = db.match_list_2016_5x5.remove({ "_id": str(list_summoners.index[0])})
result = db.match_list_2016_5x5.insert_one({ "_id": str(list_summoners.index[0]), "matches": json.loads(r.text)} )
print result.inserted_id

In [ ]:
#check result in the DB
#cursor = db.match_list_2016.find({"_id": str(list_summoners.index[0])})
#for document in cursor:
#    print(document)

In [12]:
#let's get all matches for each summoner in SEASON 2016
i = 1
while i < len(list_summoners.index):  
    ##result = db.match_list_2016_5x5.remove({ "_id": str(list_summoners.index[i])})
    #get all matches for a summoner id
    #WARNING: let's do only one request every two seconds
    r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/matchlist/by-summoner/' +str(list_summoners.index[i])+
           '?rankedQueues=TEAM_BUILDER_DRAFT_RANKED_5x5,RANKED_SOLO_5x5,RANKED_TEAM_5x5,RANKED_FLEX_SR&seasons=PRESEASON2016,SEASON2016&api_key=' + apikey)
    #check response code
    while (r.status_code <> 200):
        time.sleep(4)
        r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/matchlist/by-summoner/' +str(list_summoners.index[i])+
           '?rankedQueues=TEAM_BUILDER_DRAFT_RANKED_5x5,RANKED_SOLO_5x5,RANKED_TEAM_5x5,RANKED_FLEX_SR&seasons=PRESEASON2016,SEASON2016&api_key=' + apikey)
        
    print str(list_summoners.index[i]) +' '+ str(r.status_code)
    result = db.match_list_2016_5x5.insert_one({ "_id": str(list_summoners.index[i]), "matches": json.loads(r.text)} )
    print 'Inserted at:' + result.inserted_id
    #insert into match_list_2016(contains all matches for Season 2016 for any particular summoner) using
    #the summoner ID as DB Key    
    #increase and wait    
    i = i + 1
    time.sleep(2)

In [ ]:
#for the left behind (errors)
#r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/matchlist/by-summoner/' +'56278113'+
#           '?rankedQueues=TEAM_BUILDER_DRAFT_RANKED_5x5,RANKED_SOLO_5x5,RANKED_TEAM_3x3,RANKED_TEAM_5x5,RANKED_FLEX_SR&seasons=PRESEASON2016,SEASON2016&api_key=' + apikey)
###check response code 
#print r.status_code
#result = db.match_list_2016.remove({ "_id": '56278113'})
#result = db.match_list_2016.insert_one({ "_id": '56278113', "matches": json.loads(r.text)} )
#print result.inserted_id


In [85]:
print type (r.text)

In [86]:
print type (json.loads(r.text))